In [8]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [9]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
time="Runtime (h)"
figsize=(10,4)


# Comparison between DoRA and LoRA

1. LoRA only applied to qkv projection without ReLU
2. DoRA only applied to qkv projection without ReLU

WD=0.1 for all

In [10]:

vitc = df[df['group'].isin(["vitc-dora-ranks"])] 
# vitc["config_weight_decay"] = vitc["config_weight_decay"].astype(float)  # Ensure weight decay is float type
# vitc = vitc[vitc["config_weight_decay"] == 0.1]  # Filter for weight decay 0.1
vit_base = df[df['id'] == "2fupnc3h"] # Base model
vit_conv_stem_only = df[df['id'] == "uaryeifu"]  # Conv stem only model
vitc_base = df[df['id'] == "n8dqbqan"]  # Conv stem only model
vitc = pd.concat([vit_base, vit_conv_stem_only, vitc, vitc_base], ignore_index=True)  # Combine base and conv stem only models

vitc.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(vitc)}")

#Renaming
vitc = pu.rename(vitc).sort_values(by=val_accuracy, ascending=False)
vitc = pu.rename_adapter_factor_experiments(vitc)  # Rename columns for adapter factor experiments
vitc.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params,
    "summary__runtime": time,
}, inplace=True)

vitc.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_lora_kaiming": "ViTc LoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_dora_kaiming": "ViTc DoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_only-conv-stem": "ViTc - conv stem only",
        "vit_b_16_B=2_pretrained": "ViT",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc"
    }}, inplace=True)  # Rename specific model names for clarity

vitc["Model"] = vitc.apply(
    lambda row: f"{row['Model']} (WD={row['config_weight_decay']})", axis=1
)
vitc[time] = vitc[time].apply(lambda x: x/(60*60))  # Format runtime to 2 decimal places
# Some stats
for name in vitc["Model"].unique(): print(name)  # Print each model name
display(vitc)


Number of runs in layerwise_freezing group: 21
ViTc (WD=0.1)
ViTc DoRA (No ReLU, only qkv) (WD=0.01)
ViTc DoRA (No ReLU, only qkv) (WD=0.001)
ViTc - conv stem only (WD=0.1)
ViTc DoRA (No ReLU, only qkv) (WD=0.1)
ViT (WD=0.1)


id                                     Model     state  \
20  n8dqbqan                             ViTc (WD=0.1)  finished   
19  j6wnkdgh   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
8   8b5l32h7   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
17  5fwj40bn  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
15  upt787am   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
14  2o1sl17w   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
3   hdxefn0u   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
16  uh4esuqs   ViTc DoRA (No ReLU, only qkv) (WD=0.01)  finished   
7   la124jz5  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
11  o8a2ahn5  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
13  f16rdq4e  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
2   xm3qwmut  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
4   i3i2sahu  ViTc DoRA (No ReLU, only qkv) (WD=0.001)  finished   
1   uaryeifu            ViTc - conv stem only (WD=0.1)  finished   
10  iklv8i7m    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
12  3gwra9hk    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
5   jjeogw2n    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
9   l59n3gtw    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
6   2lgd4qop    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
18  k5iwv3v2    ViTc DoRA (No ReLU, only qkv) (WD=0.1)  finished   
0   2fupnc3h                              ViT (WD=0.1)  finished   

          entity                                 project  \
20  raphaelmaser  Improved-Interpretability-and-Concepts   
19  raphaelmaser  Improved-Interpretability-and-Concepts   
8   raphaelmaser  Improved-Interpretability-and-Concepts   
17  raphaelmaser  Improved-Interpretability-and-Concepts   
15  raphaelmaser  Improved-Interpretability-and-Concepts   
14  raphaelmaser  Improved-Interpretability-and-Concepts   
3   raphaelmaser  Improved-Interpretability-and-Concepts   
16  raphaelmaser  Improved-Interpretability-and-Concepts   
7   raphaelmaser  Improved-Interpretability-and-Concepts   
11  raphaelmaser  Improved-Interpretability-and-Concepts   
13  raphaelmaser  Improved-Interpretability-and-Concepts   
2   raphaelmaser  Improved-Interpretability-and-Concepts   
4   raphaelmaser  Improved-Interpretability-and-Concepts   
1   raphaelmaser  Improved-Interpretability-and-Concepts   
10  raphaelmaser  Improved-Interpretability-and-Concepts   
12  raphaelmaser  Improved-Interpretability-and-Concepts   
5   raphaelmaser  Improved-Interpretability-and-Concepts   
9   raphaelmaser  Improved-Interpretability-and-Concepts   
6   raphaelmaser  Improved-Interpretability-and-Concepts   
18  raphaelmaser  Improved-Interpretability-and-Concepts   
0   raphaelmaser  Improved-Interpretability-and-Concepts   

              created_at  updated_at tags notes  \
20  2025-07-19T06:49:42Z         NaN  NaN   NaN   
19  2025-07-30T13:37:28Z         NaN  NaN   NaN   
8   2025-07-30T13:36:01Z         NaN  NaN   NaN   
17  2025-07-30T22:53:57Z         NaN  NaN   NaN   
15  2025-07-30T12:17:56Z         NaN  NaN   NaN   
14  2025-07-30T13:00:37Z         NaN  NaN   NaN   
3   2025-07-30T13:37:28Z         NaN  NaN   NaN   
16  2025-07-30T13:36:01Z         NaN  NaN   NaN   
7   2025-07-30T22:54:03Z         NaN  NaN   NaN   
11  2025-07-30T22:54:03Z         NaN  NaN   NaN   
13  2025-07-30T22:54:03Z         NaN  NaN   NaN   
2   2025-07-30T22:54:03Z         NaN  NaN   NaN   
4   2025-07-30T22:58:41Z         NaN  NaN   NaN   
1   2025-07-21T10:28:29Z         NaN  NaN   NaN   
10  2025-07-29T20:43:36Z         NaN  NaN   NaN   
12  2025-07-29T20:04:13Z         NaN  NaN   NaN   
5   2025-07-29T20:43:37Z         NaN  NaN   NaN   
9   2025-07-29T19:55:02Z         NaN  NaN   NaN   
6   2025-07-29T20:48:08Z         NaN  NaN   NaN   
18  2025-07-29T20:43:28Z         NaN  NaN   NaN   
0   2025-07-19T06:48:52Z         NaN  NaN   NaN   

                                                  url            grou

In [11]:
# vitc = pd.concat([vit_c, vitc], ignore_index=True)  # Add the qkv LoRA factors to the DataFrame

## Weight decay = 0.1

In [16]:

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params,
    annotation_overlap_buffer=10
)

In [13]:
generate_scatter_plot_gridscore(
    data=vitc,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [14]:
vit_base_wd_01= vitc[vitc['config_weight_decay'] == 0.1]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=time,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params,
    range_y=(0, 7)
)